In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

/home/cdog/miniconda3/envs/ai-rmp/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'c2ee37f02fca3c6f84b3ec3c8f3ded77', 'Date': 'Thu, 22 Aug 2024 13:06:32 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [3]:
import json 
data = json.load(open('reviews.json'))

In [5]:
processed_data = []
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

for review in data['reviews']:
    response = client.embeddings.create(
        input = review['review'],
        model='text-embedding-3-small', 
    )
    embedding = response.data[0].embedding
    processed_data.append({
        'values': embedding,
        'id': review['professor'],
        'metadata': {
            'review': review['review'],
            'subject': review['subject'],
            'stars': review['stars']
        }
    })

In [6]:
processed_data[0]

{'values': [0.0010168989,
  -0.019718153,
  -0.024418486,
  0.042483777,
  0.012519155,
  -0.0004999753,
  -0.022055406,
  0.032592416,
  -0.008161017,
  -0.027866257,
  0.0063241427,
  0.008277234,
  -0.005310472,
  -0.028718516,
  0.013481174,
  0.011686266,
  0.009768685,
  -0.018723853,
  0.017264683,
  0.027375564,
  -0.00841282,
  -0.03915222,
  0.025154527,
  -0.01634786,
  -0.023695359,
  -0.056972165,
  0.012409395,
  0.034348585,
  0.032127548,
  0.02142267,
  0.050644793,
  -0.015146951,
  -0.03205007,
  -0.029906511,
  -0.03192094,
  0.079854004,
  -0.022546101,
  -0.009768685,
  -0.00595935,
  0.006701848,
  0.01695477,
  -0.045143854,
  -0.03866153,
  0.026729913,
  0.031326942,
  -0.0041224756,
  -0.030655466,
  -0.006175643,
  0.021693842,
  -0.00021831044,
  -0.004745528,
  0.013429522,
  0.01841394,
  -0.0074314326,
  -0.01651573,
  0.027633823,
  0.00214033,
  0.04199308,
  -0.037060317,
  -0.039178047,
  0.0315852,
  0.011912244,
  -0.024508877,
  -0.010097967,
  -0

In [7]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace='ns1'
)

{'upserted_count': 20}

In [8]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}